In [1]:
#1
#importing files to colab

from google.colab import files


uploaded = files.upload()

Saving Loan.csv to Loan.csv


In [2]:
import pandas as pd


In [4]:
# 1 Load data
loan = pd.read_csv('Loan.csv')
loan.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
print(loan.dtypes)

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object


In [7]:
# 2 Prepare data

# Drop "Loan_ID column"
loan.drop('Loan_ID', axis =1, inplace=True)

# Drop rows with missing data
loan.dropna(inplace=True)
loan.isna().sum()

# Create categorical features into dummy variables
loan = pd.get_dummies(loan, drop_first=True)
loan.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1,1
3,2583,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1,1
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1
5,5417,4196.0,267.0,360.0,1.0,1,1,0,1,0,0,1,0,1,1


In [8]:
# 3 Split the data into a training and test set, where the “Loan_Status” column is the target.
from sklearn.model_selection import train_test_split

x = loan.drop('Loan_Status_Y', axis=1)
Y = loan['Loan_Status_Y']

x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=0.2, random_state=42)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

# Create a pipeline with a min-max scaler and a KNN classifier
pipe_line = Pipeline([("scaler", MinMaxScaler()),("knn", KNeighborsClassifier())])

# Fit a default KNN classifier to the data with this pipeline
pipe_line.fit(x_train, Y_train)

# Accuracy of model
print(pipe_line.score(x_test, Y_test))

0.78125


In [11]:
# 7
from sklearn.model_selection import GridSearchCV

# Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10
parameter = {'knn__n_neighbors': range(1, 11)}
# parameter defines a dictionary parameter with hyperparameter knn__n_neighbors and ranges values from 1 to 10


# Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.
grid_1 = GridSearchCV(pipe_line, parameter, cv=5, n_jobs=-1, verbose=1)
#pipe_line = created in 111
# GridSearch CV is a method that performs an exhaustive search over a specified parameter grid, 
# using cross-validation to determine the best hyperparameters for a given estimator (pipe_line)
# cv=5 a parameter that specifies number of folds for cross validation
# n_jobs a parameter that specifies number of CPU cores to use for parallel proceccing. during model training
# n_jobs=-1 uses available cores
grid_1.fit(x_train, Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1, param_grid={'knn__n_neighbors': range(1, 11)},
             verbose=1)

In [12]:
# 8
# Find the accuracy of the grid search best model on the test set.
print(grid_1.score(x_test, Y_test))

0.7916666666666666


In [13]:
# Repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and 
# random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.


import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# set random seed
np.random.seed(0)

# create a pipeline
pipeline_2 = Pipeline([("classifier", RandomForestClassifier())])

# Create and expand your search space to include logistic regression and random forest models
search_space = [{
                    'classifier': [LogisticRegression()],
                    'classifier__penalty': ['l1', 'l2'],
                    'classifier__C': np.logspace(0, 4, 10)},
                {
                    'classifier': [RandomForestClassifier()],
                    'classifier__n_estimators': [10, 100, 1000],
                    'classifier__max_features': [1, 2, 3]
                }]

# create grid search
grid_2 = GridSearchCV(pipe_line, parameter, cv=5, n_jobs=-1, verbose=1)

# fit grid search
model = grid_2.fit(x_train, Y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [19]:
# What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.
# view best model
best_model = grid_2.best_estimator_
best_model
best_params = grid_2.best_params_
best_params


{'knn__n_neighbors': 3}

In [17]:
##accuracy
test_accuracy = best_model.score(x_test, Y_test)
print("Accuracy on test set:", test_accuracy)


Accuracy on test set: 0.7916666666666666


In [ ]:
# Summarize your results.
#Here are the conclusions of the study:
#The loan dataset was used to train a pipeline made up of MinMaxScaler and K-Nearest Neighbors (KNN) algorithm. On the test set, the model was correct 0.78125 of the time.
#The KNN pipeline used a grid search to find the best hyperparameter for n_neighbors, which is the amount of neighbors to look at. The search was done with 5-fold cross-validation, and n_neighbors numbers ranged from 1 to 10. With a test set accuracy of 0.7916, the best number for n_neighbors was found to be 3.
#With Random Forest and Logistic Regression classifiers, a different pipeline was made, and the models' hyperparameters were put into a search area. The grid search was then done on this process with the same number of jobs and cross-validation method. KNN with n_neighbors=3 was found to be the best model and set as its hyperparameters. On the test set, this model was 0.7916 correct.
#Overall, the models and hyperparameters tested show that the KNN classifier with n_neighbors=3 is the best model for predicting whether or not a loan will be approved. About 79% of the time, the model is right.
